In [ ]:
#working in the implementation step by step.

In [1]:
import configparser
from datetime import datetime
import os
import io
import zipfile
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, unix_timestamp
from pyspark.sql import types as t
import pandas as pd

In [2]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [3]:
with zipfile.ZipFile('song-data.zip') as zf:
    zf.extractall()
    
with zipfile.ZipFile('log-data.zip') as zf:
    zf.extractall('log-data')

In [4]:
df = spark.read.json("song_data/*/*/*/*.json")
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [5]:
df.createOrReplaceTempView("song_data")

songs_table = spark.sql("""
select song_id, title, artist_id, year, duration from song_data
""")

songs_table.limit(10).toPandas()

,song_id,title,artist_id,year,duration
0,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,ARDR4AC1187FB371A1,0,511.16363
1,SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),AREBBGV1187FB523D2,0,173.66159
2,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,ARMAC4T1187FB3FA4C,2004,207.77751
3,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,ARPBNLO1187FB3D52F,2000,43.36281
4,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,ARDNS031187B9924F0,2005,186.48771
5,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,ARNF6401187FB57032,1994,305.16200
6,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),ARLTWXK1187FB5A3F8,0,326.00771
7,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),ARPFHN61187FB575F6,0,279.97995
8,SODUJBS12A8C132150,Wessex Loses a Bride,ARI2JSK1187FB496EF,0,111.62077
9,SOBZBAZ12A6D4F8742,Spanish Grease,AROUOZZ1187B9ABE51,1997,168.25424


In [6]:
df.createOrReplaceTempView("artist_data")

artists_table = spark.sql("""
select artist_id, artist_name, artist_location, artist_latitude, artist_longitude from artist_data
""")

artists_table.limit(10).toPandas()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARDR4AC1187FB371A1,Montserrat Caballé;Placido Domingo;Vicente Sar...,,NaN,NaN
1,AREBBGV1187FB523D2,Mike Jones (Featuring CJ_ Mello & Lil' Bran),"Houston, TX",NaN,NaN
2,ARMAC4T1187FB3FA4C,The Dillinger Escape Plan,"Morris Plains, NJ",40.82624,-74.47995
3,ARPBNLO1187FB3D52F,Tiny Tim,"New York, NY",40.71455,-74.00712
4,ARDNS031187B9924F0,Tim Wilson,Georgia,32.67828,-83.22295
5,ARNF6401187FB57032,Sophie B. Hawkins,"New York, NY [Manhattan]",40.79086,-73.96644
6,ARLTWXK1187FB5A3F8,King Curtis,"Fort Worth, TX",32.74863,-97.32925
7,ARPFHN61187FB575F6,Lupe Fiasco,"Chicago, IL",41.88415,-87.63241
8,ARI2JSK1187FB496EF,Nick Ingman;Gavyn Wright,"London, England",51.50632,-0.12714
9,AROUOZZ1187B9ABE51,Willie Bobo,"New York, NY [Spanish Harlem]",40.79195,-73.94512


In [7]:
df_log_data = spark.read.json("log-data/*.json")
df_log_data.printSchema()
print(df_log_data.count())

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

8056


In [8]:
df_log_data_filter = df_log_data[df_log_data['page']=='NextSong']
print(df_log_data_filter.count())

6820


In [9]:
df_log_data_filter.createOrReplaceTempView("user_data")

users_table = spark.sql("""
select userId, firstName, lastName, gender, level from user_data
""")

users_table.drop_duplicates().limit(10).toPandas()

,userId,firstName,lastName,gender,level
0,57,Katherine,Gay,F,free
1,84,Shakira,Hunt,F,free
2,22,Sean,Wilson,F,free
3,52,Theodore,Smith,M,free
4,80,Tegan,Levine,F,paid
5,15,Lily,Koch,F,paid
6,37,Jordan,Hicks,F,free
7,98,Jordyn,Powell,F,free
8,48,Marina,Sutton,F,free
9,17,Makinley,Jones,F,free


In [10]:
# df with ts
#df = df_log_data_filter.withColumn("parsed_ts",convert_timestamp(df_log_data_filter['ts']) )
##df = df_log_data_filter.select(df_log_data_filter.ts.cast("timestamp").alias("ts"))
### documentation: https://stackoverflow.com/questions/49971903/converting-epoch-to-datetime-in-pyspark-data-frame-using-udf
df_log_data_filter = df_log_data_filter.withColumn('start_time', date_format((df_log_data_filter.ts/1000.0).cast(dataType=t.TimestampType()), "yyyy-MM-dd HH:mm:ss"))
df_log_data_filter.limit(10).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,start_time
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:30:26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:41:21
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:45:41
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,2018-11-15 03:44:09
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:48:55
5,Magic Sam,Logged In,Tegan,F,3,Levine,132.04853,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Call Me If You Need Me,200,1542261224796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:53:44
6,Edward Sharpe & The Magnetic Zeros,Logged In,Tegan,F,4,Levine,306.31138,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Home,200,1542261356796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:55:56
7,Usher featuring will.i.am,Logged In,Tegan,F,5,Levine,395.72853,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,OMG,200,1542261662796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 06:01:02
8,Helen Reddy,Logged In,Tegan,F,7,Levine,176.50893,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Candle On The Water,200,1542262057796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 06:07:37
9,Taylor Swift,Logged In,Tegan,F,8,Levine,201.06404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Our Song,200,1542262233796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 06:10:33


In [11]:
df_log_data_filter = df_log_data_filter.withColumn("hour", hour(df_log_data_filter['start_time']))
df_log_data_filter = df_log_data_filter.withColumn("day",dayofmonth(df_log_data_filter['start_time']) )
df_log_data_filter = df_log_data_filter.withColumn("week",weekofyear(df_log_data_filter['start_time']))
df_log_data_filter = df_log_data_filter.withColumn("month",month(df_log_data_filter['start_time']))
df_log_data_filter = df_log_data_filter.withColumn("year",year(df_log_data_filter['start_time']))

#df_time = df.select(
#    year(col("parsed_ts")).alias('year'), 
#    month("parsed_ts").alias('month'), 
#    dayofmonth("parsed_ts").alias('day')
#)

#convert_hour = udf(lambda z:datetime.fromtimestamp(z / 1000.0).hour)

#df = df.withColumn("hour",convert_hour(df['parsed_ts']) )
df_log_data_filter.limit(10).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,status,ts,userAgent,userId,start_time,hour,day,week,month,year
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:30:26,0,15,46,11,2018
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:41:21,0,15,46,11,2018
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:45:41,0,15,46,11,2018
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,...,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,2018-11-15 03:44:09,3,15,46,11,2018
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,...,200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:48:55,5,15,46,11,2018
5,Magic Sam,Logged In,Tegan,F,3,Levine,132.04853,paid,"Portland-South Portland, ME",PUT,...,200,1542261224796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:53:44,5,15,46,11,2018
6,Edward Sharpe & The Magnetic Zeros,Logged In,Tegan,F,4,Levine,306.31138,paid,"Portland-South Portland, ME",PUT,...,200,1542261356796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:55:56,5,15,46,11,2018
7,Usher featuring will.i.am,Logged In,Tegan,F,5,Levine,395.72853,paid,"Portland-South Portland, ME",PUT,...,200,1542261662796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 06:01:02,6,15,46,11,2018
8,Helen Reddy,Logged In,Tegan,F,7,Levine,176.50893,paid,"Portland-South Portland, ME",PUT,...,200,1542262057796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 06:07:37,6,15,46,11,2018
9,Taylor Swift,Logged In,Tegan,F,8,Levine,201.06404,paid,"Portland-South Portland, ME",PUT,...,200,1542262233796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 06:10:33,6,15,46,11,2018


In [12]:
df_log_data_filter.createOrReplaceTempView("time_data")

time_table = spark.sql("""
select start_time, hour, day, week, month, year  from time_data
""")

time_table.limit(10).toPandas()

,start_time,hour,day,week,month,year
0,2018-11-15 00:30:26,0,15,46,11,2018
1,2018-11-15 00:41:21,0,15,46,11,2018
2,2018-11-15 00:45:41,0,15,46,11,2018
3,2018-11-15 03:44:09,3,15,46,11,2018
4,2018-11-15 05:48:55,5,15,46,11,2018
5,2018-11-15 05:53:44,5,15,46,11,2018
6,2018-11-15 05:55:56,5,15,46,11,2018
7,2018-11-15 06:01:02,6,15,46,11,2018
8,2018-11-15 06:07:37,6,15,46,11,2018
9,2018-11-15 06:10:33,6,15,46,11,2018


In [13]:
# write songs table to parquet files partitioned by year and artist
#songs_table.write.partitionBy('year','artist_id').parquet("output-songs")

# write artists table to parquet files
#artists_table.write.parquet("output-artists")

# write users table to parquet files
#users_table.write.parquet("output-users")

# write time table to parquet files partitioned by year and month
#time_table.write.partitionBy('year','month')




In [18]:
    input_data = ""
    # get filepath to log data file
    log_data = input_data + "log-data/*.json"

    # read log data file
    df = spark.read.json(log_data)
    
    # filter by actions for song plays
    df = df[df['page']=='NextSong']

    df.createOrReplaceTempView("user_data")
    # extract columns for users table    
    users_table = spark.sql("""
    select userId, firstName, lastName, gender, level from user_data
    """)
    
    # write users table to parquet files
    #users_table.write.parquet("output-users")

    # create timestamp column from original timestamp column 
    # create datetime column from original timestamp column
    
    df = df.withColumn('start_time', date_format((df.ts/1000.0).cast(dataType=t.TimestampType()), "yyyy-MM-dd HH:mm:ss"))
    
    df = df.withColumn("hour", hour(df['start_time']))
    df = df.withColumn("day",dayofmonth(df['start_time']) )
    df = df.withColumn("week",weekofyear(df['start_time']))
    df = df.withColumn("month",month(df['start_time']))
    df = df.withColumn("year",year(df['start_time']))
    
    df.createOrReplaceTempView("time_data")
    
    # extract columns to create time table
    time_table = spark.sql("""
    select start_time, hour, day, week, month, year  from time_data
    """)
    
    # write time table to parquet files partitioned by year and month
    #time_table.write.partitionBy('year','month').parquet("output-time")

    # read in song data to use for songplays table
    song_df = spark.read.json(input_data + "song_data/*/*/*/*.json")
    song_df.createOrReplaceTempView("song_table")
    
    
    
    df = df.withColumn("songplay_id",monotonically_increasing_id())
    df.createOrReplaceTempView("songplays_table")
    
    
     # extract columns from joined song and log datasets to create songplays table 
    songplays_table = spark.sql("""
    select spt.songplay_id, spt.start_time, spt.userId, spt.level, spt.sessionId, spt.location, spt.userAgent,
    spt.year, spt.month, spt.length, st.song_id, st.artist_id, st.title, st.artist_name, st.duration
    from songplays_table as spt join song_table as st
    on spt.artist = st.artist_name
    """)
    
    songplays_table.limit(10).toPandas()

    # write songplays table to parquet files partitioned by year and month
    ## songplays_table
    
    

,songplay_id,start_time,userId,level,sessionId,location,userAgent,year,month,length,song_id,artist_id,title,artist_name,duration
0,392,2018-11-15 20:32:47,44,paid,619,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,2018,11,265.89995,SOBONFF12A6D4F84D8,ARIK43K1187B9AE54C,Tonight Will Be Alright,Lionel Richie,307.38240
1,882,2018-11-21 21:56:47,15,paid,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,269.58322,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,Setanta matins,Elena,269.58322
2,1187,2018-11-14 13:11:26,34,free,495,"Milwaukee-Waukesha-West Allis, WI",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,2018,11,273.94567,SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,Streets On Fire (Explicit Album Version),Lupe Fiasco,279.97995
3,1282,2018-11-14 20:16:39,101,free,603,"New Orleans-Metairie, LA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",2018,11,216.76363,SORRZGD12A6310DBC3,ARVBRGZ1187FB4675A,Harajuku Girls,Gwen Stefani,290.55955
4,1651,2018-11-28 23:22:57,24,paid,984,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",2018,11,312.92036,SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,Streets On Fire (Explicit Album Version),Lupe Fiasco,279.97995
5,3352,2018-11-24 03:48:21,88,paid,888,"Sacramento--Roseville--Arden-Arcade, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",2018,11,323.39546,SONWXQJ12A8C134D94,ARNF6401187FB57032,The Ballad Of Sleeping Beauty,Sophie B. Hawkins,305.16200
6,3466,2018-11-24 12:42:57,80,paid,903,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",2018,11,301.68771,SOBONFF12A6D4F84D8,ARIK43K1187B9AE54C,Tonight Will Be Alright,Lionel Richie,307.38240
7,3905,2018-11-29 20:29:23,49,paid,1041,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,2018,11,310.69995,SOBONFF12A6D4F84D8,ARIK43K1187B9AE54C,Tonight Will Be Alright,Lionel Richie,307.38240
8,4154,2018-11-19 15:36:04,49,paid,724,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,2018,11,154.17424,SOFSOCN12A8C143F5D,ARXR32B1187FB57099,Face the Ashes,Gob,209.60608
9,4191,2018-11-19 20:44:01,42,paid,632,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",2018,11,263.23546,SOFFKZS12AB017F194,ARBEBBY1187B9B43DB,A Higher Place (Album Version),Tom Petty,236.17261


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,status,ts,userAgent,userId,start_time,hour,day,week,month,year
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:30:26,0,15,46,11,2018
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:41:21,0,15,46,11,2018
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:45:41,0,15,46,11,2018
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,...,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,2018-11-15 03:44:09,3,15,46,11,2018
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,...,200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:48:55,5,15,46,11,2018
5,Magic Sam,Logged In,Tegan,F,3,Levine,132.04853,paid,"Portland-South Portland, ME",PUT,...,200,1542261224796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:53:44,5,15,46,11,2018
6,Edward Sharpe & The Magnetic Zeros,Logged In,Tegan,F,4,Levine,306.31138,paid,"Portland-South Portland, ME",PUT,...,200,1542261356796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:55:56,5,15,46,11,2018
7,Usher featuring will.i.am,Logged In,Tegan,F,5,Levine,395.72853,paid,"Portland-South Portland, ME",PUT,...,200,1542261662796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 06:01:02,6,15,46,11,2018
8,Helen Reddy,Logged In,Tegan,F,7,Levine,176.50893,paid,"Portland-South Portland, ME",PUT,...,200,1542262057796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 06:07:37,6,15,46,11,2018
9,Taylor Swift,Logged In,Tegan,F,8,Levine,201.06404,paid,"Portland-South Portland, ME",PUT,...,200,1542262233796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 06:10:33,6,15,46,11,2018
